## 1. Setup and Data Preparation

In [1]:

# Import necessary libraries
import os
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Enable inline plotting (for Jupyter/Colab)
%matplotlib inline


# Download DIV2K dataset


In [2]:
# Create a directory for DIV2K and switch to it
!mkdir -p /content/div2k
%cd /content/div2k

# Download DIV2K validation high-resolution images
print("Downloading DIV2K Validation HR images...")
!wget -q --show-progress http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

# Download DIV2K validation bicubic LR images (scale ×4)
print("Downloading DIV2K Validation LR images (bicubic ×4)...")
!wget -q --show-progress http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip

# Extract the downloaded zip files
print("Extracting images...")
!unzip -q DIV2K_valid_HR.zip
!unzip -q DIV2K_valid_LR_bicubic_X4.zip

print("Download and extraction complete.")


/content/div2k
DIV2K_valid_HR.zip  100%[===================>] 428.19M  30.6MB/s    in 15s     
DIV2K_valid_LR_bicu 100%[===================>]  30.05M  18.6MB/s    in 1.6s    
Extracting images...
Download and extraction complete.


In [3]:
# List HR and LR files
hr_dir = "DIV2K_valid_HR"
lr_dir = "DIV2K_valid_LR_bicubic/X4"

hr_files = sorted(glob.glob(os.path.join(hr_dir, "*.png")))
lr_files = sorted(glob.glob(os.path.join(lr_dir, "*.png")))

print(f"Found {len(hr_files)} HR images")
print(f"Found {len(lr_files)} bicubic LR images")


Found 100 HR images
Found 100 bicubic LR images


In [4]:
def bicubic_upscale(lr_image, scale=4):
    """Upscale a PIL image by the given scale factor using bicubic interpolation."""
    hr_size = (lr_image.width * scale, lr_image.height * scale)
    return lr_image.resize(hr_size, Image.BICUBIC)

# Test upscaling on one image to verify shapes
sample_lr = Image.open(lr_files[0])
upscaled = bicubic_upscale(sample_lr, scale=4)
print(f"Sample LR size: {sample_lr.size}, Upscaled size: {upscaled.size}")


Sample LR size: (510, 339), Upscaled size: (2040, 1356)


In [5]:
psnr_values = []
ssim_values = []

for lr_path, hr_path in zip(lr_files, hr_files):
    # Load images
    lr_img = Image.open(lr_path)
    hr_img = Image.open(hr_path)
    # Upscale LR by factor 4 using bicubic
    up_img = bicubic_upscale(lr_img, scale=4)
    # Convert to NumPy arrays (dtype=uint8)
    hr_arr = np.array(hr_img)
    up_arr = np.array(up_img)
    # Compute PSNR and SSIM (specify data_range=255 for 8-bit images)
    psnr_val = peak_signal_noise_ratio(hr_arr, up_arr, data_range=255)
    ssim_val = structural_similarity(hr_arr, up_arr, data_range=255, win_size=7, channel_axis=-1)
    psnr_values.append(psnr_val)
    ssim_values.append(ssim_val)

print(f"Computed PSNR and SSIM for {len(psnr_values)} image pairs.")
print(f"Average PSNR: {np.mean(psnr_values):.2f} dB; Average SSIM: {np.mean(ssim_values):.4f}")


Computed PSNR and SSIM for 100 image pairs.
Average PSNR: 26.70 dB; Average SSIM: 0.7663


In [6]:
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# blurry LR from HR
def simulate_blurry_lr(hr_img, scale=4):
    small = hr_img.resize((hr_img.width // scale, hr_img.height // scale), Image.BICUBIC)
    small = small.filter(ImageFilter.GaussianBlur(radius=2.5))
    return small

# Bicubic Upscaler
def bicubic_upscale(lr_img, scale=4):
    new_size = (lr_img.width * scale, lr_img.height * scale)
    return lr_img.resize(new_size, Image.BICUBIC)

# Plot triplet: Simulated LR, Bicubic Upscale, HR
def plot_image_triplet_from_hr(hr_path, scale=4):
    # Load HR
    hr_img = Image.open(hr_path).convert("RGB")

    # Simulate blurry LR from HR
    lr_img = simulate_blurry_lr(hr_img, scale)

    # Upscale LR back to HR size using bicubic
    bicubic_img = bicubic_upscale(lr_img, scale)

    # Convert to numpy
    lr_np = np.array(lr_img)
    bicubic_np = np.array(bicubic_img)
    hr_np = np.array(hr_img)

    # Compute PSNR & SSIM
    psnr_val = peak_signal_noise_ratio(hr_np, bicubic_np, data_range=255)
    ssim_val = structural_similarity(hr_np, bicubic_np, channel_axis=-1, data_range=255)

    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(lr_np)
    axes[0].set_title(f"LR\n{lr_np.shape}")
    axes[0].axis("off")

    axes[1].imshow(bicubic_np)
    axes[1].set_title(f"Bicubic Upscale\nPSNR: {psnr_val:.2f} | SSIM: {ssim_val:.3f}")
    axes[1].axis("off")

    axes[2].imshow(hr_np)
    axes[2].set_title(f"High-Res (GT)\n{hr_np.shape}")
    axes[2].axis("off")

    plt.tight_layout()
    plt.show()

# Example: visualize 3 samples from HR file list
for i in range(3):
    plot_image_triplet_from_hr(hr_files[i])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for hr_path in hr_files:
    hr_img = Image.open(hr_path).convert("RGB")

    # Simulate LR from HR
    lr_img = simulate_blurry_lr(hr_img, scale=4)

    # Upscale back to HR resolution
    up_img = lr_img.resize(hr_img.size, Image.BICUBIC)

    # Convert to NumPy
    hr_arr = np.array(hr_img)
    up_arr = np.array(up_img)

    # Compute metrics
    psnr_val = peak_signal_noise_ratio(hr_arr, up_arr, data_range=255)
    ssim_val = structural_similarity(hr_arr, up_arr, channel_axis=2, data_range=255)

    psnr_values.append(psnr_val)
    ssim_values.append(ssim_val)

print(f"Computed PSNR and SSIM for {len(psnr_values)} image pairs.")
print(f"Average PSNR: {np.mean(psnr_values):.2f} dB; Average SSIM: {np.mean(ssim_values):.4f}")


Computed PSNR and SSIM for 200 image pairs.
Average PSNR: 23.65 dB; Average SSIM: 0.6472


# Mathematical implementation of bicubic interpolation

The pure mathematical implementation of bicubic interpolation can be really slow.

In [ ]:
import os
import random
import cv2
import numpy as np
import math
import sys, time

# -----------------------------
# Interpolation kernel function
def u(s, a):
    if (abs(s) >= 0) & (abs(s) <= 1):
        return (a + 2) * (abs(s)**3) - (a + 3) * (abs(s)**2) + 1
    elif (abs(s) > 1) & (abs(s) <= 2):
        return a * (abs(s)**3) - (5 * a) * (abs(s)**2) + (8 * a) * abs(s) - 4 * a
    return 0

# -----------------------------
# Padding the image
def padding(img, H, W, C):
    zimg = np.zeros((H+4, W+4, C))
    zimg[2:H+2, 2:W+2, :C] = img
    # Pad sides
    zimg[2:H+2, 0:2, :C] = img[:, 0:1, :C]
    zimg[H+2:H+4, 2:W+2, :] = img[H-1:H, :, :]
    zimg[2:H+2, W+2:W+4, :] = img[:, W-1:W, :]
    zimg[0:2, 2:W+2, :C] = img[0:1, :, :C]
    # Pad corners
    zimg[0:2, 0:2, :C] = img[0,0,:C]
    zimg[H+2:H+4, 0:2, :C] = img[H-1,0,:C]
    zimg[H+2:H+4, W+2:W+4, :C] = img[H-1,W-1,:C]
    zimg[0:2, W+2:W+4, :C] = img[0,W-1,:C]
    return zimg

# -----------------------------
# Progress bar
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

# -----------------------------
# Manual Bicubic Interpolation
def bicubic(img, ratio, a):
    H, W, C = img.shape
    img = padding(img, H, W, C)

    dH = math.floor(H * ratio)
    dW = math.floor(W * ratio)
    dst = np.zeros((dH, dW, 3))

    h = 1 / ratio

    print('Start bicubic interpolation')
    print('It will take a little while...')
    inc = 0
    for c in range(C):
        for j in range(dH):
            for i in range(dW):
                x, y = i * h + 2, j * h + 2

                x1 = 1 + x - math.floor(x)
                x2 = x - math.floor(x)
                x3 = math.floor(x) + 1 - x
                x4 = math.floor(x) + 2 - x

                y1 = 1 + y - math.floor(y)
                y2 = y - math.floor(y)
                y3 = math.floor(y) + 1 - y
                y4 = math.floor(y) + 2 - y

                mat_l = np.matrix([[u(x1, a), u(x2, a), u(x3, a), u(x4, a)]])
                mat_m = np.matrix([
                    [img[int(y-y1), int(x-x1), c], img[int(y-y2), int(x-x1), c], img[int(y+y3), int(x-x1), c], img[int(y+y4), int(x-x1), c]],
                    [img[int(y-y1), int(x-x2), c], img[int(y-y2), int(x-x2), c], img[int(y+y3), int(x-x2), c], img[int(y+y4), int(x-x2), c]],
                    [img[int(y-y1), int(x+x3), c], img[int(y-y2), int(x+x3), c], img[int(y+y3), int(x+x3), c], img[int(y+y4), int(x+x3), c]],
                    [img[int(y-y1), int(x+x4), c], img[int(y-y2), int(x+x4), c], img[int(y+y3), int(x+x4), c], img[int(y+y4), int(x+x4), c]]
                ])
                mat_r = np.matrix([[u(y1, a)], [u(y2, a)], [u(y3, a)], [u(y4, a)]])
                dst[j, i, c] = np.dot(np.dot(mat_l, mat_m), mat_r)

                # Print progress
                inc += 1
                sys.stderr.write('\r\033[K' + get_progressbar_str(inc / (C * dH * dW)))
                sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()

    return dst

# -----------------------------
# Main program

# Dataset folder path
dataset_path = '/content/datasets/DIV2K_valid_HR/'
  # <<< change this to your dataset folder

# List all .png files
all_files = os.listdir(dataset_path)
image_files = [f for f in all_files if f.endswith('.png')]

# Randomly select one image
selected_file = random.choice(image_files)
img_path = os.path.join(dataset_path, selected_file)

# Read selected image
img = cv2.imread(img_path)

if img is None:
    print(" Error: Could not load image. Check the path.")
    exit()
else:
    print(f" Loaded image: {selected_file}")

# Parameters
ratio = 2  # Upscale 2x
a = -0.5   # Cubic interpolation coefficient (common value)

# Bicubic interpolation
dst = bicubic(img, ratio, a)

print(' Completed bicubic interpolation!')

# Save result
output_filename = f'bicubic_{selected_file}'
cv2.imwrite(output_filename, dst)
print(f' Saved upscaled image as: {output_filename}')



<ipython-input-2-716212428874>:82: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dst[j, i, c] = np.dot(np.dot(mat_l, mat_m), mat_r)
Progress:[>                              ] 0.0%

✅ Loaded image: 0841.png
Start bicubic interpolation
It will take a little while...


Progress:[>                              ] 0.2%